# Biolink Metamodel Test Notebook

In [1]:
!pip -q --disable-pip-version-check install yamlmagic
%reload_ext yamlmagic

In [2]:
from IPython.display import display, HTML
from types import ModuleType

from jsonasobj import as_json, loads
from rdflib import Graph

from linkml.generators.jsonldcontextgen import ContextGenerator
from linkml.generators.pythongen import PythonGenerator
from linkml.generators.shexgen import ShExGenerator
from linkml.generators.yumlgen import YumlGenerator
from linkml_runtime.utils.yamlutils import DupCheckYamlLoader
from linkml_runtime.dumpers import json_dumper

## Basic model structure
A biolink model consists of:
* a name
* a uri
* type definitions
* slot definitions
* class definitions
* subset definitions

As an example, the model below defines:

In [3]:
%%yaml  model
id: http://example.org/sample/example1
name: synopsis2
prefixes:
    foaf: http://xmlns.com/foaf/0.1/
    samp: http://example.org/model/
    xsd: http://www.w3.org/2001/XMLSchema#

default_prefix: samp

default_curi_maps:
    - semweb_context

default_range: string

types:
    string:
        base: str
        uri: xsd:string
    int:
        base: int
        uri: xsd:integer
    boolean:
        base: Bool
        uri: xsd:boolean


classes:
    person:
        description: A person, living or dead
        slots:
            - id
            - first name
            - last name
            - age
            - living
            - knows

    friendly_person:
        description: Any person that knows someone
        is_a: person
        slot_usage:
            knows:
                required: True

slots:
    id:
        description: Unique identifier of a person
        identifier: true

    first name:
        description: The first name of a person
        slot_uri: foaf:firstName
        multivalued: true

    last name:
        description: The last name of a person
        slot_uri: foaf:lastName
        required: true

    living:
        description: Whether the person is alive
        range: boolean
        comments:
            - unspecified means unknown

    age:
        description: The age of a person if living or age of death if not
        range: int
        slot_uri: foaf:age

    knows:
        description: A person known by this person (indicating some level of reciprocated interaction between the parties).
        range: person
        slot_uri: foaf:knows
        multivalued: true

<IPython.core.display.Javascript object>

### We can emit this model as a Python class

In [4]:
import yaml
model_str = yaml.dump(model)
print(PythonGenerator(model_str, gen_slots=False).serialize())

# Auto generated from None by pythongen.py version: 0.0.1
# Generation date: 2025-01-09T17:12:55
# Schema: synopsis2
#
# id: http://example.org/sample/example1
# description:
# license:

import dataclasses
import re
from dataclasses import dataclass
from datetime import (
    date,
    datetime,
    time
)
from typing import (
    Any,
    ClassVar,
    Dict,
    List,
    Optional,
    Union
)

from jsonasobj2 import (
    JsonObj,
    as_dict
)
from linkml_runtime.linkml_model.meta import (
    EnumDefinition,
    PermissibleValue,
    PvFormulaOptions
)
from linkml_runtime.utils.curienamespace import CurieNamespace
from linkml_runtime.utils.dataclass_extensions_376 import dataclasses_init_fn_with_kwargs
from linkml_runtime.utils.enumerations import EnumDefinitionImpl
from linkml_runtime.utils.formatutils import (
    camelcase,
    sfx,
    underscore
)
from linkml_runtime.utils.metamodelcore import (
    bnode,
    empty_dict,
    empty_list
)
from linkml_runtime.utils.slot import 

### Compile the python into a module

In [5]:
spec = compile(PythonGenerator(model_str).serialize(), 'test', 'exec')
module = ModuleType('test')
exec(spec, module.__dict__)

### We can emit a UML rendering of  this model

In [6]:
display(HTML(f'<img src="{YumlGenerator(model_str).serialize()}"/>'))

### We can emit a JSON-LD context for the model:

In [7]:
cntxt = ContextGenerator(model_str).serialize(base="http://example.org/people/")
print(cntxt)

{
   "comments": {
      "description": "Auto generated by LinkML jsonld context generator",
      "generation_date": "2025-01-09T17:12:57",
      "source": null
   },
   "@context": {
      "xsd": "http://www.w3.org/2001/XMLSchema#",
      "foaf": "http://xmlns.com/foaf/0.1/",
      "samp": "http://example.org/model/",
      "@vocab": "http://example.org/model/",
      "age": {
         "@type": "xsd:integer",
         "@id": "foaf:age"
      },
      "first_name": {
         "@id": "foaf:firstName"
      },
      "id": "@id",
      "knows": {
         "@type": "@id",
         "@id": "foaf:knows"
      },
      "last_name": {
         "@id": "foaf:lastName"
      },
      "living": {
         "@type": "xsd:boolean",
         "@id": "living"
      },
      "@base": "http://example.org/people/",
      "FriendlyPerson": {
         "@id": "FriendlyPerson"
      },
      "Person": {
         "@id": "Person"
      }
   }
}



### The python model can be used to create classes

In [8]:
# Generate a person
joe_smith = module.Person(id="42", last_name="smith", first_name=['Joe', 'Bob'], age=43)
print(joe_smith)

Person({'id': '42', 'last_name': 'smith', 'first_name': ['Joe', 'Bob'], 'age': 43})


### and can be combined w/ the JSON-LD Context to generate RDF

In [9]:
# Add the context and turn it into RDF
jsonld = json_dumper.dumps(joe_smith, cntxt)
print(jsonld)
g = Graph()
g.parse(data=jsonld, format="json-ld")
print(g.serialize(format="turtle"))

{
  "id": "42",
  "last_name": "smith",
  "first_name": [
    "Joe",
    "Bob"
  ],
  "age": 43,
  "@type": "Person",
  "@context": {
    "xsd": "http://www.w3.org/2001/XMLSchema#",
    "foaf": "http://xmlns.com/foaf/0.1/",
    "samp": "http://example.org/model/",
    "@vocab": "http://example.org/model/",
    "age": {
      "@type": "xsd:integer",
      "@id": "foaf:age"
    },
    "first_name": {
      "@id": "foaf:firstName"
    },
    "id": "@id",
    "knows": {
      "@type": "@id",
      "@id": "foaf:knows"
    },
    "last_name": {
      "@id": "foaf:lastName"
    },
    "living": {
      "@type": "xsd:boolean",
      "@id": "living"
    },
    "@base": "http://example.org/people/",
    "FriendlyPerson": {
      "@id": "FriendlyPerson"
    },
    "Person": {
      "@id": "Person"
    }
  }
}
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix samp: <http://example.org/model/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://example.org/people/42> a samp:Person ;
  

### The model can be turned into ShEx

In [10]:
shex = ShExGenerator(model_str).serialize(collections=False)
print(shex)

# metamodel_version: 1.7.0
BASE <http://example.org/model/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


<Boolean> xsd:boolean

<Int> xsd:integer

<String> xsd:string

<FriendlyPerson> CLOSED {
    (  $<FriendlyPerson_tes> (  &<Person_tes> ;
          rdf:type [ <Person> ] ? ;
          foaf:knows @<Person> +
       ) ;
       rdf:type [ <FriendlyPerson> ]
    )
}

<Person>  (
    CLOSED {
       (  $<Person_tes> (  foaf:firstName @<String> * ;
             foaf:lastName @<String> ;
             foaf:age @<Int> ? ;
             <living> @<Boolean> ? ;
             foaf:knows @<Person> *
          ) ;
          rdf:type [ <Person> ]
       )
    } OR @<FriendlyPerson>
)




### The ShEx can then be used to validate RDF

In [11]:
from pyshex.evaluate import evaluate
r = evaluate(g, shex,
             start="http://example.org/model/Person",
             focus="http://example.org/people/42")
print("Conforms" if r[0] else r[1])

Conforms


In [12]:
r = evaluate(g, shex,
             start="http://example.org/model/FriendlyPerson",
             focus="http://example.org/people/42")
print("Conforms" if r[0] else r[1])

Conforms
